# Flight Price ML - 데이터 전처리 파이프라인

이 노트북은 항공권 가격 예측을 위한 전체 전처리 파이프라인을 포함합니다:

1. **데이터 로딩 및 탐색**
2. **데이터 정제 (중복 제거)**
3. **Outlier 분석 및 처리**
4. **Feature Engineering**: 원본 데이터 → Feature Schema 형식
5. **ML Preprocessing**: Feature → 모델 입력 형식 (인코딩/스케일링)
6. **데이터 저장 및 검증**

## Feature Schema
- `purchase_day_of_week`: 구매 요일 (one-hot)
- `purchase_time_bucket`: 구매 시간대 (one-hot)
- `days_until_departure_bucket`: 출발까지 남은 일수 (ordinal)
- `is_weekend_departure`: 주말 출발 여부 (boolean)
- `is_holiday_season`: 휴가 성수기 여부 (boolean)
- `price_trend_7d`: 가격 추세 (numeric)
- `current_vs_historical_avg`: 현재/평균 가격 비율 (numeric)
- `route_hash`: 경로 해시 (categorical)
- `stops_count`: 경유 횟수 (numeric)
- `flight_duration_bucket`: 비행 시간 구간 (one-hot)
- **Target**: `price`

## 1. 환경 설정 및 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import hashlib
import warnings
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

print("✅ 라이브러리 임포트 완료")

## 2. 데이터 로딩 및 탐색

In [ ]:
# 데이터 로드
df_raw = pd.read_csv('data/raw/flight_fares.csv')

print(f"데이터셋 크기: {df_raw.shape}")
print(f"\n컬럼 목록:")
print(df_raw.columns.tolist())

In [ ]:
# 데이터 미리보기
df_raw.head()

In [ ]:
# 데이터 타입 및 결측치 확인
print("데이터 타입:")
print(df_raw.dtypes)
print("\n결측치:")
print(df_raw.isnull().sum())

In [ ]:
# 기본 통계
df_raw.describe()

In [ ]:
# 중복 데이터 확인
duplicates = df_raw.duplicated().sum()
duplicate_pct = (duplicates / len(df_raw)) * 100

print(f"🔍 중복 데이터 분석:")
print(f"  - 총 데이터: {len(df_raw):,}개")
print(f"  - 중복 데이터: {duplicates:,}개")
print(f"  - 중복 비율: {duplicate_pct:.2f}%")

if duplicates > 0:
    print(f"\n⚠️ {duplicates:,}개의 중복 데이터가 발견되었습니다. 다음 단계에서 제거합니다.")

In [ ]:
# 가격 분포 시각화 (초기)
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Histogram
axes[0, 0].hist(df_raw['Fare'], bins=50, edgecolor='black')
axes[0, 0].set_title('Fare Distribution (Original)')
axes[0, 0].set_xlabel('Fare')
axes[0, 0].set_ylabel('Frequency')

# Box plot
axes[0, 1].boxplot(df_raw['Fare'])
axes[0, 1].set_title('Fare Box Plot')
axes[0, 1].set_ylabel('Fare')

# Log scale histogram
axes[1, 0].hist(np.log1p(df_raw['Fare']), bins=50, edgecolor='black')
axes[1, 0].set_title('Log(Fare) Distribution')
axes[1, 0].set_xlabel('Log(Fare)')
axes[1, 0].set_ylabel('Frequency')

# Q-Q plot
stats.probplot(df_raw['Fare'], dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot (Normal Distribution)')

plt.tight_layout()
plt.show()

# Skewness 계산
skewness = df_raw['Fare'].skew()
print(f"\n📊 가격 통계:")
print(f"  - Skewness: {skewness:.2f}")
if abs(skewness) > 1:
    print(f"  ⚠️ Highly skewed (|skew| > 1)")
elif abs(skewness) > 0.5:
    print(f"  ⚠️ Moderately skewed (|skew| > 0.5)")
print(f"\n{df_raw['Fare'].describe()}")

## 3. 데이터 정제 (중복 제거)

In [ ]:
# 중복 데이터 제거
df_raw_before = df_raw.shape[0]
df_raw = df_raw.drop_duplicates()
df_raw_after = df_raw.shape[0]
removed = df_raw_before - df_raw_after

print(f"✅ 중복 제거 완료")
print(f"  - 제거 전: {df_raw_before:,}개")
print(f"  - 제거 후: {df_raw_after:,}개")
print(f"  - 제거된 데이터: {removed:,}개 ({(removed/df_raw_before)*100:.2f}%)")

# 인덱스 리셋
df_raw = df_raw.reset_index(drop=True)
print(f"\n최종 데이터셋 크기: {df_raw.shape}")

## 4. Outlier 분석 및 처리

가격 데이터는 skewed되어 있고 outlier가 많습니다. 여러 처리 방법을 제공합니다.

In [ ]:
def detect_outliers_iqr(data, column, multiplier=1.5):
    """
    IQR 방법으로 outlier 탐지
    
    Args:
        data: DataFrame
        column: 컬럼명
        multiplier: IQR 배수 (기본 1.5, 더 엄격하게는 3.0)
    
    Returns:
        lower_bound, upper_bound, outlier_mask
    """
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    
    outlier_mask = (data[column] < lower_bound) | (data[column] > upper_bound)
    
    return lower_bound, upper_bound, outlier_mask

# Outlier 탐지
lower, upper, outlier_mask = detect_outliers_iqr(df_raw, 'Fare', multiplier=1.5)
n_outliers = outlier_mask.sum()
outlier_pct = (n_outliers / len(df_raw)) * 100

print(f"🔍 Outlier 분석 (IQR 방법, multiplier=1.5):")
print(f"  - Lower bound: ₹{lower:,.0f}")
print(f"  - Upper bound: ₹{upper:,.0f}")
print(f"  - Outliers: {n_outliers:,}개 ({outlier_pct:.2f}%)")
print(f"  - Min outlier: ₹{df_raw[outlier_mask]['Fare'].min():,.0f}")
print(f"  - Max outlier: ₹{df_raw[outlier_mask]['Fare'].max():,.0f}")

In [ ]:
# 시각화: Outlier 분포
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot
axes[0].scatter(range(len(df_raw)), df_raw['Fare'], 
                c=outlier_mask, cmap='coolwarm', alpha=0.5, s=1)
axes[0].axhline(y=upper, color='r', linestyle='--', label=f'Upper bound: ₹{upper:,.0f}')
axes[0].axhline(y=lower, color='r', linestyle='--', label=f'Lower bound: ₹{lower:,.0f}')
axes[0].set_title('Fare Distribution with Outliers')
axes[0].set_xlabel('Index')
axes[0].set_ylabel('Fare')
axes[0].legend()

# Violin plot
axes[1].violinplot([df_raw[~outlier_mask]['Fare'], df_raw[outlier_mask]['Fare']], 
                    positions=[1, 2], showmeans=True)
axes[1].set_xticks([1, 2])
axes[1].set_xticklabels(['Normal', 'Outliers'])
axes[1].set_ylabel('Fare')
axes[1].set_title('Normal vs Outliers Distribution')

plt.tight_layout()
plt.show()

### Outlier 처리 방법 선택

아래 3가지 옵션 중 하나를 선택하세요:

**옵션 1: Outlier 제거** - 극단값 완전 제거  
**옵션 2: Outlier Clipping (Winsorization)** - 극단값을 경계값으로 대체 (권장)  
**옵션 3: Log Transformation** - 가격에 log 변환 적용 (skewness 감소)

아래에서 하나를 실행하세요:

In [ ]:
# 옵션 설정
OUTLIER_METHOD = 'clip'  # 'remove', 'clip', 'log', 'none' 중 선택

df_processed = df_raw.copy()

if OUTLIER_METHOD == 'remove':
    # 옵션 1: Outlier 제거
    before = len(df_processed)
    df_processed = df_processed[~outlier_mask].reset_index(drop=True)
    after = len(df_processed)
    print(f"✅ Outlier 제거 완료")
    print(f"  - 제거 전: {before:,}개")
    print(f"  - 제거 후: {after:,}개")
    print(f"  - 제거된 데이터: {before - after:,}개 ({((before-after)/before)*100:.2f}%)")
    
elif OUTLIER_METHOD == 'clip':
    # 옵션 2: Outlier Clipping (권장)
    df_processed['Fare'] = df_processed['Fare'].clip(lower=lower, upper=upper)
    print(f"✅ Outlier Clipping 완료")
    print(f"  - Lower bound: ₹{lower:,.0f}")
    print(f"  - Upper bound: ₹{upper:,.0f}")
    print(f"  - {n_outliers:,}개의 값이 경계값으로 대체되었습니다")
    
elif OUTLIER_METHOD == 'log':
    # 옵션 3: Log Transformation
    # Note: Feature engineering 후 target에 적용해야 함
    print(f"✅ Log Transformation 선택")
    print(f"  - Feature engineering 후 target에 log1p 변환을 적용합니다")
    print(f"  - 모델 예측 후 expm1으로 역변환 필요")
    
else:
    # 옵션 4: 처리 안 함
    print(f"ℹ️ Outlier 처리 없음 - 원본 데이터 사용")

print(f"\n최종 데이터 크기: {df_processed.shape}")
print(f"\n처리 후 가격 통계:")
print(df_processed['Fare'].describe())

In [ ]:
# 처리 후 분포 확인
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 원본 vs 처리 후 비교
axes[0].hist(df_raw['Fare'], bins=50, alpha=0.5, label='Original', edgecolor='black')
axes[0].hist(df_processed['Fare'], bins=50, alpha=0.5, label='Processed', edgecolor='black')
axes[0].set_title('Fare Distribution: Before vs After')
axes[0].set_xlabel('Fare')
axes[0].set_ylabel('Frequency')
axes[0].legend()

# Box plot 비교
axes[1].boxplot([df_raw['Fare'], df_processed['Fare']], labels=['Original', 'Processed'])
axes[1].set_title('Box Plot Comparison')
axes[1].set_ylabel('Fare')

# Skewness 개선 확인
original_skew = df_raw['Fare'].skew()
processed_skew = df_processed['Fare'].skew()
axes[2].bar(['Original', 'Processed'], [original_skew, processed_skew])
axes[2].axhline(y=0, color='r', linestyle='--')
axes[2].set_title('Skewness Comparison')
axes[2].set_ylabel('Skewness')

plt.tight_layout()
plt.show()

print(f"\n📊 Skewness 개선:")
print(f"  - Original: {original_skew:.3f}")
print(f"  - Processed: {processed_skew:.3f}")
print(f"  - Improvement: {((original_skew - processed_skew) / original_skew * 100):.1f}%")

## 5. Feature Engineering

처리된 데이터를 Feature Schema에 맞게 변환합니다.

In [ ]:
class FlightFeatureEngineer:
    """
    원본 데이터를 feature schema에 정의된 feature로 변환
    """
    
    def __init__(self, apply_log_to_target=False):
        # 인도 공휴일 시즌 정의
        self.holiday_months = [1, 3, 4, 5, 6, 8, 10, 11]  # Republic Day, Holi, 여름휴가, Independence Day, Diwali/Dussehra
        self.apply_log_to_target = apply_log_to_target
        
    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        원본 데이터를 feature schema 형식으로 변환
        """
        df = df.copy()
        
        # 날짜/시간 파싱
        df['crawl_datetime'] = pd.to_datetime(df['Crawl Timestamp'], utc=True).dt.tz_localize(None)
        df['departure_datetime'] = pd.to_datetime(df['Departure Date'] + ' ' + df['Departure Time'])
        
        features = pd.DataFrame()
        
        # 1. purchase_day_of_week: 구매(크롤링) 시점의 요일
        features['purchase_day_of_week'] = df['crawl_datetime'].dt.dayofweek
        
        # 2. purchase_time_bucket: 구매 시간대
        features['purchase_time_bucket'] = df['crawl_datetime'].dt.hour.apply(
            self._get_time_bucket
        )
        
        # 3. days_until_departure_bucket: 출발까지 남은 일수
        days_until = (df['departure_datetime'] - df['crawl_datetime']).dt.days
        features['days_until_departure_bucket'] = days_until.apply(
            self._get_days_until_bucket
        )
        
        # 4. is_weekend_departure: 주말 출발 여부
        features['is_weekend_departure'] = (
            df['departure_datetime'].dt.dayofweek >= 5
        ).astype(int)
        
        # 5. is_holiday_season: 휴가 성수기 여부
        features['is_holiday_season'] = (
            df['departure_datetime'].dt.month.isin(self.holiday_months)
        ).astype(int)
        
        # 6. price_trend_7d: 가격 추세 (route별 평균 대비)
        features['price_trend_7d'] = self._calculate_price_trend(df)
        
        # 7. current_vs_historical_avg: 현재 가격 vs 평균 가격 비율
        features['current_vs_historical_avg'] = self._calculate_price_ratio(df)
        
        # 8. route_hash: 출발지-목적지 해시
        features['route_hash'] = df.apply(
            lambda row: self._hash_route(row['Source'], row['Destination']),
            axis=1
        )
        
        # 9. stops_count: 경유 횟수
        features['stops_count'] = df['Number Of Stops']
        
        # 10. flight_duration_bucket: 비행 시간 구간
        total_minutes = df['Total Time'].apply(self._parse_duration)
        features['flight_duration_bucket'] = total_minutes.apply(
            self._get_duration_bucket
        )
        
        # Target: price (optional log transform)
        if self.apply_log_to_target:
            features['price'] = np.log1p(df['Fare'])
            features['price_original'] = df['Fare']  # 원본 보존
        else:
            features['price'] = df['Fare']
        
        return features
    
    def _get_time_bucket(self, hour: int) -> str:
        """시간을 bucket으로 변환"""
        if 0 <= hour < 6:
            return 'dawn'
        elif 6 <= hour < 12:
            return 'morning'
        elif 12 <= hour < 18:
            return 'afternoon'
        else:
            return 'night'
    
    def _get_days_until_bucket(self, days: int) -> str:
        """출발까지 남은 일수를 bucket으로 변환"""
        if days < 7:
            return 'very_close'
        elif days < 14:
            return 'close'
        elif days < 30:
            return 'medium'
        else:
            return 'far'
    
    def _hash_route(self, source: str, destination: str) -> str:
        """출발지-목적지를 해시값으로 변환"""
        route_str = f"{source}_{destination}"
        return hashlib.md5(route_str.encode()).hexdigest()[:8]
    
    def _parse_duration(self, duration_str: str) -> int:
        """
        비행 시간 문자열을 분으로 변환
        예: "2h 30m" -> 150
        """
        try:
            if pd.isna(duration_str):
                return 0
            
            hours = 0
            minutes = 0
            
            if 'h' in str(duration_str):
                parts = str(duration_str).split('h')
                hours = int(parts[0].strip())
                if len(parts) > 1 and 'm' in parts[1]:
                    minutes = int(parts[1].replace('m', '').strip())
            elif 'm' in str(duration_str):
                minutes = int(str(duration_str).replace('m', '').strip())
            
            return hours * 60 + minutes
        except:
            return 0
    
    def _get_duration_bucket(self, minutes: int) -> str:
        """비행 시간을 bucket으로 변환"""
        if minutes < 120:  # 2시간 미만
            return 'short'
        elif minutes < 360:  # 6시간 미만
            return 'medium'
        else:
            return 'long'
    
    def _calculate_price_trend(self, df: pd.DataFrame) -> pd.Series:
        """가격 추세 계산 (route별 평균 대비 변화율)"""
        df['route'] = df['Source'] + '_' + df['Destination']
        route_avg = df.groupby('route')['Fare'].transform('mean')
        trend = (df['Fare'] - route_avg) / route_avg
        return trend.fillna(0)
    
    def _calculate_price_ratio(self, df: pd.DataFrame) -> pd.Series:
        """현재 가격 vs 평균 가격 비율"""
        df['route'] = df['Source'] + '_' + df['Destination']
        route_avg = df.groupby('route')['Fare'].transform('mean')
        ratio = df['Fare'] / route_avg
        return ratio.fillna(1.0)

print("✅ FlightFeatureEngineer 클래스 정의 완료")

In [ ]:
# Feature Engineering 실행
# apply_log_to_target=True로 설정하면 가격에 log 변환 적용
engineer = FlightFeatureEngineer(apply_log_to_target=(OUTLIER_METHOD == 'log'))
df_features = engineer.transform(df_processed)

print(f"✅ Feature Engineering 완료")
print(f"변환된 Feature 개수: {df_features.shape[1]}")
print(f"\nFeature 목록:")
print(df_features.columns.tolist())

if OUTLIER_METHOD == 'log':
    print(f"\n⚠️ Target에 log 변환이 적용되었습니다.")
    print(f"   모델 예측 후 np.expm1()으로 역변환해야 합니다.")

In [ ]:
# 변환된 데이터 미리보기
df_features.head(10)

In [ ]:
# Feature Engineering 후 중복 확인
duplicates_after_fe = df_features.duplicated().sum()
duplicate_pct_fe = (duplicates_after_fe / len(df_features)) * 100

print(f"🔍 Feature Engineering 후 중복 데이터 분석:")
print(f"  - 총 데이터: {len(df_features):,}개")
print(f"  - 중복 데이터: {duplicates_after_fe:,}개")
print(f"  - 중복 비율: {duplicate_pct_fe:.2f}%")

if duplicates_after_fe > 0:
    print(f"\n⚠️ {duplicates_after_fe:,}개의 중복 데이터가 발견되었습니다.")

In [ ]:
# Feature Engineering 후 중복 제거
if duplicates_after_fe > 0:
    df_features_before = df_features.shape[0]
    df_features = df_features.drop_duplicates()
    df_features = df_features.reset_index(drop=True)
    df_features_after = df_features.shape[0]
    removed_fe = df_features_before - df_features_after
    
    print(f"✅ Feature Engineering 후 중복 제거 완료")
    print(f"  - 제거 전: {df_features_before:,}개")
    print(f"  - 제거 후: {df_features_after:,}개")
    print(f"  - 제거된 데이터: {removed_fe:,}개 ({(removed_fe/df_features_before)*100:.2f}%)")
else:
    print("✅ 중복 없음")

print(f"\n최종 Feature 데이터셋 크기: {df_features.shape}")

In [ ]:
# Feature 분포 확인
print("Feature 통계:")
print(df_features.describe())

print("\n결측치 확인:")
print(df_features.isnull().sum())

In [ ]:
# 카테고리 feature 분포 확인
categorical_features = ['purchase_time_bucket', 'days_until_departure_bucket', 'flight_duration_bucket']

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, col in enumerate(categorical_features):
    df_features[col].value_counts().plot(kind='bar', ax=axes[idx])
    axes[idx].set_title(f'{col} Distribution')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Count')

plt.tight_layout()
plt.show()

## 6. Train/Validation/Test Split (70/10/20)

In [ ]:
# Target과 features 분리
target_col = 'price'
X = df_features.drop([c for c in df_features.columns if 'price' in c], axis=1)
y = df_features[target_col]

# 먼저 Train(70%) / Temp(30%) 분할
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Temp를 Validation(10%) / Test(20%)로 분할
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=2/3, random_state=42
)

print(f"✅ 데이터셋 분할 완료 (Train 70% / Validation 10% / Test 20%)")
print(f"\n📊 데이터셋 크기:")
print(f"  - 전체: {len(X):,}개")
print(f"  - Train: {X_train.shape[0]:,}개 ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  - Validation: {X_val.shape[0]:,}개 ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"  - Test: {X_test.shape[0]:,}개 ({X_test.shape[0]/len(X)*100:.1f}%)")

price_label = 'Log(Price)' if OUTLIER_METHOD == 'log' else 'Price'
print(f"\n💰 {price_label} 통계:")
print(f"  - Train 평균: {y_train.mean():.2f}")
print(f"  - Validation 평균: {y_val.mean():.2f}")
print(f"  - Test 평균: {y_test.mean():.2f}")

## 7. ML Preprocessing (인코딩 & 스케일링)

Feature를 ML 모델 입력 형식으로 변환합니다.

In [ ]:
class FlightPricePreprocessor:
    """
    Feature schema 기반 전처리 클래스
    - Categorical: One-hot encoding
    - Ordinal: Label encoding
    - Numeric: Standardization (선택)
    - Boolean: 0/1 (그대로)
    """
    
    def __init__(self, scale_numeric: bool = True):
        self.scale_numeric = scale_numeric
        self.preprocessor = None
        self.ordinal_mapping = {
            'very_close': 0,
            'close': 1,
            'medium': 2,
            'far': 3
        }
        self._setup_preprocessor()
        
    def _setup_preprocessor(self):
        """전처리 파이프라인 설정"""
        
        # Feature 그룹 정의
        categorical_onehot_features = [
            'purchase_day_of_week',
            'purchase_time_bucket', 
            'flight_duration_bucket'
        ]
        
        boolean_features = [
            'is_weekend_departure',
            'is_holiday_season'
        ]
        
        numeric_features = [
            'price_trend_7d',
            'current_vs_historical_avg',
            'stops_count'
        ]
        
        high_cardinality_features = ['route_hash']
        
        # ColumnTransformer 구성
        transformers = []
        
        # 1. Categorical (one-hot)
        transformers.append((
            'cat_onehot',
            OneHotEncoder(
                drop='first',
                sparse_output=False,
                handle_unknown='ignore'
            ),
            categorical_onehot_features
        ))
        
        # 2. Numeric features
        if self.scale_numeric:
            transformers.append((
                'num',
                StandardScaler(),
                numeric_features
            ))
        else:
            transformers.append((
                'num',
                'passthrough',
                numeric_features
            ))
        
        # 3. Boolean features (그대로)
        transformers.append((
            'bool',
            'passthrough',
            boolean_features
        ))
        
        # 4. High cardinality (그대로)
        transformers.append((
            'high_card',
            'passthrough',
            high_cardinality_features
        ))
        
        self.preprocessor = ColumnTransformer(
            transformers=transformers,
            remainder='drop'
        )
        
    def _encode_ordinal_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Ordinal feature 인코딩"""
        df = df.copy()
        if 'days_until_departure_bucket' in df.columns:
            df['days_until_departure_bucket'] = df['days_until_departure_bucket'].map(
                self.ordinal_mapping
            )
        return df
    
    def fit(self, X: pd.DataFrame, y=None):
        """전처리기를 학습 데이터에 fit"""
        X_processed = self._encode_ordinal_features(X)
        self.preprocessor.fit(X_processed)
        return self
    
    def transform(self, X: pd.DataFrame) -> np.ndarray:
        """데이터를 전처리하여 변환"""
        X_processed = self._encode_ordinal_features(X)
        X_transformed = self.preprocessor.transform(X_processed)
        return X_transformed
    
    def fit_transform(self, X: pd.DataFrame, y=None) -> np.ndarray:
        """fit과 transform을 한 번에 수행"""
        return self.fit(X, y).transform(X)
    
    def get_feature_names_out(self):
        """변환 후 feature 이름 반환"""
        try:
            return list(self.preprocessor.get_feature_names_out())
        except AttributeError:
            return ['feature_' + str(i) for i in range(self.preprocessor.transform(X_train.head(1)).shape[1])]

print("✅ FlightPricePreprocessor 클래스 정의 완료")

In [ ]:
# Preprocessing 실행
preprocessor = FlightPricePreprocessor(scale_numeric=True)

# Train 데이터로 fit & transform
X_train_processed = preprocessor.fit_transform(X_train)

# Validation & Test 데이터 transform
X_val_processed = preprocessor.transform(X_val)
X_test_processed = preprocessor.transform(X_test)

print(f"✅ ML Preprocessing 완료")
print(f"원본 feature 수: {X_train.shape[1]}")
print(f"변환 후 feature 수: {X_train_processed.shape[1]}")
print(f"\n📊 데이터셋 크기:")
print(f"  - Train: {X_train_processed.shape}")
print(f"  - Validation: {X_val_processed.shape}")
print(f"  - Test: {X_test_processed.shape}")

In [ ]:
# 변환된 feature 이름 확인
feature_names = preprocessor.get_feature_names_out()
print(f"Feature 이름 ({len(feature_names)}개):")
for i, name in enumerate(feature_names, 1):
    print(f"{i:2d}. {name}")

In [ ]:
# 변환된 데이터 미리보기
X_train_df = pd.DataFrame(X_train_processed, columns=feature_names)
X_train_df.head()

## 8. 데이터 저장

전처리된 데이터를 저장합니다.

In [ ]:
import os

# 디렉토리 생성
os.makedirs('data/processed', exist_ok=True)

# Feature engineered 데이터 저장 (CSV)
df_features.to_csv('data/processed/features.csv', index=False)
print("✅ Feature engineered data 저장 완료: data/processed/features.csv")

# 전처리된 데이터 저장 (numpy)
np.save('data/processed/X_train.npy', X_train_processed)
np.save('data/processed/X_val.npy', X_val_processed)
np.save('data/processed/X_test.npy', X_test_processed)
np.save('data/processed/y_train.npy', y_train.values)
np.save('data/processed/y_val.npy', y_val.values)
np.save('data/processed/y_test.npy', y_test.values)
print("✅ Preprocessed data 저장 완료: data/processed/*.npy")

# Feature 이름 저장
with open('data/processed/feature_names.txt', 'w') as f:
    for name in feature_names:
        f.write(name + '\n')
print("✅ Feature names 저장 완료: data/processed/feature_names.txt")

# 메타데이터 저장 (outlier 처리 방법 등)
metadata = {
    'outlier_method': OUTLIER_METHOD,
    'log_transformed': (OUTLIER_METHOD == 'log'),
    'outlier_bounds': {'lower': float(lower), 'upper': float(upper)}
}
import json
with open('data/processed/metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
print("✅ Metadata 저장 완료: data/processed/metadata.json")

## 9. 요약 및 검증

In [ ]:
print("=" * 70)
print("전처리 파이프라인 완료")
print("=" * 70)

print(f"\n📊 데이터 정제:")
print(f"  - 중복 제거: {removed:,}개")
print(f"  - Outlier 처리: {OUTLIER_METHOD}")
if OUTLIER_METHOD == 'clip':
    print(f"    └ Clipped: {n_outliers:,}개")
elif OUTLIER_METHOD == 'remove':
    print(f"    └ Removed: {n_outliers:,}개")
print(f"  - Skewness: {original_skew:.2f} → {processed_skew:.2f}")

print(f"\n📊 데이터셋 정보:")
print(f"  - 최종 데이터: {len(df_processed):,}개")
print(f"  - Feature engineered: {df_features.shape}")
print(f"  - Train set: {X_train_processed.shape}")
print(f"  - Validation set: {X_val_processed.shape}")
print(f"  - Test set: {X_test_processed.shape}")

print(f"\n🎯 Feature 정보:")
print(f"  - 원본 feature 수: {X_train.shape[1]}")
print(f"  - One-hot encoding 후: {X_train_processed.shape[1]}")

print(f"\n💰 Target (가격) 정보:")
if OUTLIER_METHOD == 'log':
    print(f"  ⚠️ Log-transformed target")
    print(f"  - Train 평균 (log): {y_train.mean():.2f}")
    print(f"  - Val 평균 (log): {y_val.mean():.2f}")
    print(f"  - Test 평균 (log): {y_test.mean():.2f}")
else:
    print(f"  - Train 평균: ₹{y_train.mean():,.0f}")
    print(f"  - Validation 평균: ₹{y_val.mean():,.0f}")
    print(f"  - Test 평균: ₹{y_test.mean():,.0f}")
    print(f"  - 전체 범위: ₹{y.min():,.0f} ~ ₹{y.max():,.0f}")

print(f"\n📁 저장된 파일:")
print(f"  - data/processed/features.csv")
print(f"  - data/processed/X_train.npy")
print(f"  - data/processed/X_val.npy")
print(f"  - data/processed/X_test.npy")
print(f"  - data/processed/y_train.npy")
print(f"  - data/processed/y_val.npy")
print(f"  - data/processed/y_test.npy")
print(f"  - data/processed/feature_names.txt")
print(f"  - data/processed/metadata.json")

print(f"\n✅ 다음 단계: 모델 학습 및 평가")

## 10. 데이터 로드 헬퍼 함수

나중에 데이터를 쉽게 로드할 수 있는 함수

In [ ]:
def load_processed_data():
    """
    저장된 전처리 데이터 로드
    
    Returns:
        X_train, X_val, X_test, y_train, y_val, y_test, feature_names, metadata
    """
    import json
    
    X_train = np.load('data/processed/X_train.npy')
    X_val = np.load('data/processed/X_val.npy')
    X_test = np.load('data/processed/X_test.npy')
    y_train = np.load('data/processed/y_train.npy')
    y_val = np.load('data/processed/y_val.npy')
    y_test = np.load('data/processed/y_test.npy')
    
    with open('data/processed/feature_names.txt', 'r') as f:
        feature_names = [line.strip() for line in f]
    
    with open('data/processed/metadata.json', 'r') as f:
        metadata = json.load(f)
    
    return X_train, X_val, X_test, y_train, y_val, y_test, feature_names, metadata

print("✅ load_processed_data() 함수 정의 완료")
print("   사용법: X_train, X_val, X_test, y_train, y_val, y_test, feature_names, metadata = load_processed_data()")
print("\n⚠️ 중요: Log transformation을 사용한 경우 예측값을 np.expm1()으로 역변환해야 합니다!")